In [16]:
import cv2
import numpy as np
img_width=580
img_height=640
cam=cv2.VideoCapture(r'D:\Image Processing with OpenCV (1)\Image Processing with OpenCV\OpenCV\Resources\OCR.mp4')
cam.set(3,640)
cam.set(4,480)

False

In [17]:
def Preprocessing(img):
    imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray,(5,5),1)
    imgCanny = cv2.Canny(imgBlur,90,90)
    kernel = np.ones((5,5))
    imgDial = cv2.dilate(imgCanny,kernel,iterations=2)
    imgThres = cv2.erode(imgDial,kernel,iterations=1)
    return imgThres

In [18]:
def getContours(img):
    biggest = np.array([])
    maxArea = 0
    contours,hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)[-2:] # define image, retrieval method:external:extreme outer contours, approx : none, all countours
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area>5000:
            #cv2.drawContours(imgContour, cnt, -1, (255, 0, 0), 3)
            peri = cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,0.02*peri,True)
            if area >maxArea and len(approx) == 4:
                biggest = approx
                maxArea = area
    cv2.drawContours(imgContour, biggest, -1, (255, 0, 0), 20)
    return biggest

In [19]:
def reorder (myPoints):
    myPoints = myPoints.reshape((4,2))
    myPointsNew = np.zeros((4,1,2),np.int32)
    add = myPoints.sum(1)
    #print("add", add)
    myPointsNew[0] = myPoints[np.argmin(add)] # smallest corner
    myPointsNew[3] = myPoints[np.argmax(add)] # biggest corner
    diff = np.diff(myPoints,axis=1)
    myPointsNew[1]= myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
    #print("NewPoints",myPointsNew)
    return myPointsNew
def getWarp(img,biggest):
    biggest = reorder(biggest)
    pts1 = np.float32(biggest)
    pts2 = np.float32([[0, 0], [img_width, 0], [0, img_height], [img_width,img_height]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgOutput = cv2.warpPerspective(img, matrix, (img_width, img_height))
    imgCropped = imgOutput[20:imgOutput.shape[0]-20,20:imgOutput.shape[1]-20]
    imgCropped = cv2.resize(imgCropped,(img_width,img_height))
    return imgCropped


In [20]:
while True:
    success, img = cam.read()
    img = cv2.resize(img,(img_width,img_height))
    imgContour = img.copy()
    imgThres = Preprocessing(img)
    biggest = getContours(imgThres)
    if biggest.size !=0:
        imgWarped=getWarp(img,biggest)
        cv2.imshow("ImageWarped", imgWarped)

    cv2.imshow("ImageThreshold",imgThres)
    cv2.imshow("ImageContour", imgContour)

    if cv2.waitKey(4000) & 0xFF == ord('q'):
        break
cam.release()
cv2.destroyAllWindows()
